# COVID-19 Visualization

In [1]:
import pandas as pd
import numpy as np
#import seaborn as sns
#from matplotlib import pyplot as plt
#import plotly.graph_objects as go
#import plotly.express as px

# Data Import, Preprocessing

In [2]:
df = pd.read_csv('../data/raw/covid_19_data.csv',parse_dates=['Last Update'])
df.rename(columns={'ObservationDate':'Date', 'Country/Region':'Country'}, inplace=True)

In [3]:
confirmed_grp = df.groupby('Date').sum()['Confirmed'].reset_index()
deaths_grp = df.groupby('Date').sum()['Deaths'].reset_index()
recovered_grp = df.groupby('Date').sum()['Recovered'].reset_index()

confirmed_grp['shift'] = confirmed_grp['Confirmed'].shift()
confirmed_grp['increased'] = confirmed_grp['Confirmed']-confirmed_grp['shift']
confirmed_y = confirmed_grp['increased'].to_numpy()

deaths_grp['shift'] = deaths_grp['Deaths'].shift()
deaths_grp['increased'] = deaths_grp['Deaths']-deaths_grp['shift']
deaths_y = deaths_grp['increased'].to_numpy()

recovered_grp['shift'] = recovered_grp['Recovered'].shift()
recovered_grp['increased'] = recovered_grp['Recovered']-recovered_grp['shift']
recovered_y = recovered_grp['increased'].to_numpy()

dates = np.arange('2020-01-22', '2020-03-19', dtype='datetime64[D]')

# Start Vuetify

In [4]:
from typing import List
from IPython.display import display, clear_output
import ipywidgets as widgets
import ipyvuetify as v
import traitlets
from traitlets import Int, Float, Unicode, Bool
import bqplot.pyplot as plt
from bqplot import DateScale

# Model

In [5]:
class Model(traitlets.HasTraits):
    confirmed = Bool(True)
    deaths = Bool(False)
    recovered = Bool(False)

model = Model()


# View

In [6]:
confirmed_checkbox = v.Checkbox(v_model=False, label='Confirmed')
deaths_checkbox = v.Checkbox(v_model=False, label='Deaths')
recovered_checkbox = v.Checkbox(v_model=False, label='Recovered')

# Scenario

In [7]:


def scenario(confirmed = False, deaths= False, recovered = False):
    
    # Report input assumptions
    scenario_conditions = [
        f'confirmed: {"yes" if confirmed else "no"}',
        f'deaths: {"yes" if deaths else "no"}',
        f'recovered: {"yes" if recovered else "no"}',
        
        
    ]
    fig = plt.figure(title='Worldwide Corona Virus Cases - Confirmed, Deaths, Recovered', 
                )
    if confirmed:
        plt.plot(dates, confirmed_grp.Confirmed,  #axes_options=axes_options,
                'g-.s', labels=['Confirmed']) # third argument is the marker string      
        
    if deaths:
        plt.plot(dates, deaths_grp.Deaths,  #axes_options=axes_options,
                'm:d',labels=['Deaths'])
    if recovered:
        plt.plot(dates, recovered_grp.Recovered,  #axes_options=axes_options,
                'b-.s',labels=['Recovered']) 
    plt.xlabel('Date')
    plt.ylabel('Number of Cases')
    plt.legend()
    selector = plt.brush_int_selector()
    
    
    return fig, scenario_conditions, selector

In [8]:
y = []
colors = []

def create_grp(confirmed = False, deaths= False, recovered = False):
    fig_hist = plt.figure()
    y.clear()
    colors.clear()
    if confirmed:
        y.append(confirmed_y)
        colors.append('green')
    if deaths:
        y.append(deaths_y)
        colors.append('magenta')
    if recovered:
        y.append(recovered_y)
        colors.append('blue')
  
    plt.bar(dates,y,padding=0.2, type='grouped',colors=colors)
  
    return fig_hist

def update_grp(mask):
    new_y = [e[mask] for e in y]
    plt.clear()
    bar = plt.bar(dates[mask],new_y,padding=0.2, type='grouped',colors=colors)

# Layout

In [9]:
def make_card(title="Title", children=[]):
    card_content = v.CardText(children=children)
    return (
        v.Card(outlined=True, tile=True, #class_='ma-1 pa-2', 
            children=[
                v.CardTitle(children=[title]),
                card_content
            ]),
        card_content
    )

def make_control(title, children):
    return (
        v.Card(flat=True, #class_='ma-1 pa-2', 
            children=[
                v.CardText(children=[title]),
                v.CardText(children=children)
            ])
        )

# To be used in the presentation section
def make_infobox(scenario_conditions):
    return [
        v.List(children=[
            v.ListItem(children=[line])
            for line in scenario_conditions
        ])
    ]


cc =  make_control('Confirmed', [confirmed_checkbox])
dc = make_control('Deaths', [deaths_checkbox])
rc = make_control('Recovered', [recovered_checkbox])

conditions_infobox, cic = make_card('Scenario assumptions')
covid_chart_box, ccb = make_card('Worldwide Corona Virus Cases - Confirmed, Deaths, Recovered', [widgets.Output()])
grp_chart_box, gcb = make_card('Group Bar Chart', [widgets.Output()])

In [10]:
center_content = v.Col(dense=True, children=[
            v.Row(children=[
                v.Col(cols="3", children=[
                    cc
                ]),
                  v.Col(cols="3", children=[
                    dc
                ]),
                  v.Col(cols="3", children=[
                    rc
                ])
            ]),
            v.Row(children=[
                v.Col(cols="12", children=[
                    covid_chart_box
                ])
            ]),
            v.Row(children=[
                v.Col(cols="12", children=[
                    grp_chart_box
                ])
            ]),
            v.Row(children=[
                v.Col(cols="12", children=[
                    conditions_infobox
                ])
            ])
])


app = v.Container(_metadata={'mount_id': 'content-main'}, children=[
     center_content
])


# Render

In [11]:
# selector = None
def render_scenario(change):
    def update_range(*ignore):
        if selector.selected is not None and len(selector.selected) == 2:
            xmin, xmax = selector.selected
            mask = (dates > xmin) & (dates < xmax)
            update_grp(mask)
            
    with model.hold_trait_notifications():
        (
          fig, 
          scenario_conditions, 
          selector
        ) = scenario(
                confirmed = model.confirmed,
                deaths  = model.deaths,
                recovered  = model.recovered,
                )    
       
       
        cic.children = make_infobox(scenario_conditions)
        with ccb.children[0]:
            clear_output(wait=True)
            display(fig)
           
        
        with gcb.children[0]:
            clear_output(wait=True)
            fig_hist = create_grp(confirmed = model.confirmed,deaths = model.deaths,recovered = model.recovered)
            display(fig_hist)
            selector.observe(update_range, 'selected')  

            
model.observe(render_scenario, traitlets.All)


In [12]:
traitlets.link(
    (model, 'confirmed'), (confirmed_checkbox, 'v_model')
);

traitlets.link(
    (model, 'deaths'), (deaths_checkbox, 'v_model')
);
traitlets.link(
    (model, 'recovered'), (recovered_checkbox, 'v_model')
);

In [13]:
render_scenario('')

app


Container(children=[Col(children=[Row(children=[Col(children=[Card(children=[CardText(children=['Confirmed']),…